In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.preprocessing import image
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, smart_resize
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.constraints import maxnorm
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import cv2
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
from keras.metrics import AUC

In [ ]:
train_folder = '../input/plant-pathology-2021-fgvc8/train_images'
test_folder =  '../input/plant-pathology-2021-fgvc8/test_images'

In [ ]:
df_train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df_test = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')

In [ ]:
df_train.describe()

In [ ]:
df_train.head()

In [ ]:
df_train.labels.unique()

In [ ]:
labels = df_train.labels.unique()

In [ ]:
df_train.labels.value_counts()

In [ ]:
df_train.labels = df_train.labels.astype('category')

In [ ]:
train_datagen  = ImageDataGenerator(
                    rotation_range = 180,
                    shear_range = 0.3,
                    height_shift_range = 0.1,
                    horizontal_flip = True,
                    rescale = 1./255,
                    zoom_range = 0.2,
                    validation_split = 0.2)
test_datagen  = ImageDataGenerator(
                    rescale = 1./255,
                    validation_split = 0.2)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe = df_train,
                           directory = train_folder,
                           target_size = (150,150),
                           x_col = 'image',
                           y_col = 'labels',
                           batch_size = 32,
                           class_mode = 'categorical',
                           subset = 'training')

validator_generator = test_datagen.flow_from_dataframe(dataframe = df_train,
                             directory = train_folder,
                             target_size = (150,150),
                             x_col = 'image',
                             y_col = 'labels',
                             batch_size = 32,
                             class_mode = 'categorical',
                             subset = 'validation')

In [ ]:
'''model = Sequential()

model.add(Conv2D(32, (3, 3), padding = 'same', input_shape= (150,150,3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(12))
model.add(Activation('softmax'))'''

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding="same", activation="relu", input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(12, activation="softmax"))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

cambiar metrica metrics=['accuracy', ADC()]

In [ ]:
#history = model.fit(train_generator, steps_per_epoch=10, epochs=5, validation_data=validator_generator, validation_steps=50)
#history = model.fit(train_generator, epochs=25, validation_data=validator_generator) 
#history = model.fit(train_generator, steps_per_epoch=10, epochs=10, validation_data=validator_generator, validation_steps=50)
#history = model.fit(train_generator, steps_per_epoch=10, epochs=10, validation_data=validator_generator, validation_steps=100)

In [ ]:
model.fit(train_generator, steps_per_epoch=10, epochs=10, validation_data=validator_generator, validation_steps=50)

In [ ]:
scores = model.evaluate(validator_generator, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model.save('model.h5')
model.save_weights('weights.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(10)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


PREDICTIONS

In [ ]:

model = './model.h5'
weight_model = './weights.h5'


cnn = load_model(model)
cnn.load_weights(weight_model)

test_generator = test_datagen.flow_from_dataframe(dataframe = df_test,
                             directory = test_folder,
                             target_size = (150,150),
                             x_col = 'image',
                             y_col = 'labels',
                             batch_size = 32,
                             class_mode = 'categorical')


predictions=(cnn.predict(test_generator))

In [ ]:
predictions

In [ ]:
for l in labels:
    print(l)

In [ ]:
max_values = []
pred = []
for p in predictions:
    pred.append(p)
    max_values.append(max(p))

In [ ]:
max_values

In [ ]:
l = []
for value in pred:
    for i, v in enumerate(value):
        if v in max_values:
            print(i,v)
            l.append(i)

In [ ]:
label_final = []
for i, label in enumerate(labels):
    for j in l:
        if i == j:
            label_final.append(label)
print(label_final)

In [ ]:
df_test['final_label'] = label_final

In [ ]:
df_test.to_csv('submission.csv', index=False)

In [ ]:
df_test